# Robust vehicle pre-allocation

In [35]:
import cvxpy as cp
import scipy as sc
import numpy as np
import numpy.random as npr
import torch
from sklearn import datasets
import pandas as pd
import lropt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes

In [36]:
import pandas as pd
data = pd.read_csv('https://xiongpengnus.github.io/rsome/taxi_rain.csv')
demand = data.loc[:, 'Region1':'Region10']  
data = demand.values                              # sample demand as an array
d_ub = demand.max().values                      # upper bound of demand
d_lb = demand.min().values 
# S = dhat.shape[0]  
I, J = 1, 10
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])
c = 3 * np.ones(J)
q = 400 * np.ones(I)

In [37]:
u = lropt.UncertainParameter(J,
                        uncertainty_set=lropt.Ellipsoidal(p=np.inf, rho = np.max(d_ub), lb = d_lb, ub = d_ub))
# Formulate the Robust Problem
x = cp.Variable(J)
y = cp.Variable(J)
Y = cp.Variable((J,J))
objective = cp.Minimize((c-r)@x + r@y + r@Y@u)
constraints = []
for i in range(J):
    constraints += [y[i] >= x[i] - (Y[i]+np.eye(J)[i])@u]
    constraints += [y[i] + Y[i]@u >= 0]
constraints += [x >= 0, cp.sum(x)<= q]
prob = lropt.RobustProblem(objective=objective, constraints=constraints)

In [38]:
prob.solve()

-62.589735284066215